#  *Data Comparision*

from the data scraped from the Googole compare the data with the user query using simantic similarity methods

In [1]:
#! pip install pandas

In [2]:
import pandas as pd
import json

### load the universal sentence encoder from tensorflow_hub 

In [3]:
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np

In [4]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" 
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


### create the embedings for the data collected from the google 

In [5]:
#convert json to pandas dataframe 
data = pd.read_json(r'..\scrapData\data.json')
data.head(8)

,url,title,h1
0,https://en.wikipedia.org/wiki/Semantic_Web,[Semantic Web - Wikipedia],[Semantic Web]
1,https://www.researchgate.net/publication/33364...,[(PDF) Semantic Web Mining: Survey and Analysis],[Semantic Web Mining Survey and Analysis]
2,https://support.google.com/,[Google Help],[]
3,http://www.public.asu.edu/~hdavulcu/CSE591_Sem...,[CSE591: Semantic Web Mining],[]
4,https://www.researchgate.net/publication/25755...,[(PDF) Mining the Semantic Web],[Mining the Semantic Web]
5,https://www.slideshare.net/anilmlis/semantic-w...,[Semantic Web Mining],[]
6,https://s3-us-west-2.amazonaws.com/ieeeshutpag...,[IEEE Xplore is temporarily unavailable],"[IEEE , is temporarily unavailable]"


In [6]:
#d.x = d.x.apply(lambda y: np.nan if len(y)==0 else y)
data.h1 = data.h1.apply(lambda y : ["no data"] if len(y)==0 else y)

In [7]:
# h1 = data['h1']
# embeded_data = embed(data)
data['h1']

0                               [Semantic Web]
1    [Semantic Web Mining Survey and Analysis]
2                                    [no data]
3                                    [no data]
4                    [Mining the Semantic Web]
5                                    [no data]
6         [IEEE ,  is temporarily unavailable]
Name: h1, dtype: object

In [8]:
#embed(data.h1.tolist()[7])
converted_h1 = data.h1.tolist() 
for i in range(len(converted_h1)) :
    converted_h1[i] = embed(converted_h1[i])
#print(converted_h1)  

In [9]:
# add embeded text to the dataframe
data['embeded_h1'] = converted_h1

In [10]:
#read the query file
query_file = open('..\scrapData\query.txt')
query = query_file.readline()

In [11]:
#embed the search query
query_embeded = embed([query])

### Compair the simanitc of the search query and the google results

In [12]:

from sklearn.metrics.pairwise import cosine_similarity

In [13]:
# cosine similarity WRT to header tags
cos_sim = []
for i in data.index:
    
    #cosine_similarity(converted_h1[i],query_embeded)
    cos_sim.append(cosine_similarity(converted_h1[i],query_embeded))
    

In [14]:
# l = []
# for i in data.h1:
#     l.append(str(i[0]))
# l
#cummulative_cos = []
for i in range(len(cos_sim)):
    
#     cummulative_cos.append((sum(cos_sim[i][:])/len(cos_sim[i][:])))
    #print(cos_sim[i])
    cos_sim[i] = float(sum(cos_sim[i][:])/len(cos_sim[i][:]))
    #print(cos_sim[i])
#cos_sim

In [15]:
data['cosine_similarity'] = cos_sim

In [16]:
data

,url,title,h1,embeded_h1,cosine_similarity
0,https://en.wikipedia.org/wiki/Semantic_Web,[Semantic Web - Wikipedia],[Semantic Web],"((tf.Tensor(0.057723746, shape=(), dtype=float...",0.519466
1,https://www.researchgate.net/publication/33364...,[(PDF) Semantic Web Mining: Survey and Analysis],[Semantic Web Mining Survey and Analysis],"((tf.Tensor(0.058095824, shape=(), dtype=float...",0.622203
2,https://support.google.com/,[Google Help],[no data],"((tf.Tensor(-0.050970808, shape=(), dtype=floa...",0.062301
3,http://www.public.asu.edu/~hdavulcu/CSE591_Sem...,[CSE591: Semantic Web Mining],[no data],"((tf.Tensor(-0.050970808, shape=(), dtype=floa...",0.062301
4,https://www.researchgate.net/publication/25755...,[(PDF) Mining the Semantic Web],[Mining the Semantic Web],"((tf.Tensor(0.06788203, shape=(), dtype=float3...",0.760898
5,https://www.slideshare.net/anilmlis/semantic-w...,[Semantic Web Mining],[no data],"((tf.Tensor(-0.050970808, shape=(), dtype=floa...",0.062301
6,https://s3-us-west-2.amazonaws.com/ieeeshutpag...,[IEEE Xplore is temporarily unavailable],"[IEEE , is temporarily unavailable]","((tf.Tensor(-0.037304126, shape=(), dtype=floa...",0.115383


In [17]:
data.sort_values(by=['cosine_similarity'],ascending=False ,inplace=True)
data[['url','h1','cosine_similarity']]

,url,h1,cosine_similarity
4,https://www.researchgate.net/publication/25755...,[Mining the Semantic Web],0.760898
1,https://www.researchgate.net/publication/33364...,[Semantic Web Mining Survey and Analysis],0.622203
0,https://en.wikipedia.org/wiki/Semantic_Web,[Semantic Web],0.519466
6,https://s3-us-west-2.amazonaws.com/ieeeshutpag...,"[IEEE , is temporarily unavailable]",0.115383
2,https://support.google.com/,[no data],0.062301
3,http://www.public.asu.edu/~hdavulcu/CSE591_Sem...,[no data],0.062301
5,https://www.slideshare.net/anilmlis/semantic-w...,[no data],0.062301
